In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import uniform
from truncreg import Truncreg
import DEA

In [7]:

#=============================================== Monte-Carlo ===============================================
#trial
trial = 10

beta1_mean_list = []
beta1_std_list = []
beta2_mean_list = []
beta2_std_list = []
sigma_mean_list = []
sigma_std_list = []

for t in range(trial):
    #=============================================== DGP ===============================================
    # sample size
    n = 100 #400

    # beta
    beta_1 = 0.5
    beta_2 = 0.5

    # generate environment variable z
    np.random.seed(seed=3*t)
    mu_z = 2
    sigma_z = 2
    z = norm.rvs(mu_z,sigma_z,size=n)
    zn = norm.rvs(mu_z,sigma_z,size=n)

    # generate error term
    np.random.seed(seed=3*t+1)
    sigma_e = 1
    error = norm.rvs(0,sigma_e,size=n)

    # generate inefficiency U
    delta = beta_1 + z*beta_2 + error
    for i in range(n): # left-truncated
        if delta[i] < 1:
            delta[i] = 1

    # generate input x
    np.random.seed(seed=3*t+3)
    x = uniform.rvs(6,16,size=n)

    # generate output y
    y = (delta**-1) * (x**(3/4))


    #=============================================== DEA ===============================================
    # DataFrame
    DMU_index = list(range(0,n))
    DMU = [str(x) for x in DMU_index]
    df_Y = pd.DataFrame({'Y':y},index=DMU)
    df_X = pd.DataFrame({'X':x},index=DMU)
    Y = df_Y.T.to_dict('list')
    X = df_X.T.to_dict('list')

    # input-oriented VRS (dual)
    d = DEA.VRS(DMU, X, Y, orientation="output", dual=True)

    #=============================================== Truncated Regression ===============================================
    # DataFrame
    #df = pd.DataFrame({'D': d, 'D_trunc':d, 'Z':z})
    df = pd.DataFrame({'D': d, 'D_trunc':d, 'Z':z, 'Zn':zn})

    # Truncated Data
    left = 1
    cond = (df.loc[:,'D'] <= left)
    df.loc[cond,'D_trunc'] = np.nan
    d_trunc = df['D_trunc']

    # Fitting Truncated Regression
    formula_trunc = 'D_trunc ~ Z'
    res_trunc = Truncreg.from_formula(formula_trunc,left=1,data=df).fit()
    print(res_trunc.summary().tables[1])
    df_trunc = pd.read_html(res_trunc.summary().tables[1].as_html(),header=0,index_col=0)[0]
    beta1_h = df_trunc.coef[0]
    beta2_h = df_trunc.coef[1]
    sigma_h = np.mean(res_trunc.resid**2)

    #=============================================== Bootstrap Estimation ===============================================
    B = 10
    beta1_b_list = []
    beta2_b_list = []
    sigma_b_list = []
    for b in range(B):
        #1
        np.random.seed(seed=b)
        eps = norm.rvs(0, sigma_h, size=n)
        for i in range(n):
            if eps[i] < 1-z[i]*beta2_h - beta1_h:
                eps[i] = 1-z[i]*beta2_h - beta1_h
            else:
                eps[i] = eps[i]

        #2
        d_s = beta1_h + z*beta2_h + eps

        #3
        # DataFrame
        df_b = pd.DataFrame({'D': d_s, 'D_trunc': d_s, 'Z': z})

        # Truncated Data
        left = 1
        cond = (df_b.loc[:, 'D'] <= left)
        df_b.loc[cond, 'D_trunc'] = np.nan
        d_trunc = df_b['D_trunc']

        # Fitting Truncated Regression
        formula_trunc = 'D_trunc ~ Z'
        res_b_trunc = Truncreg.from_formula(formula_trunc, left=1, data=df_b).fit()
        df_b_trunc = pd.read_html(res_b_trunc.summary().tables[1].as_html(), header=0, index_col=0)[0]
        beta1_b = df_b_trunc.coef[0]
        beta2_b = df_b_trunc.coef[1]
        sigma_b = np.mean(res_b_trunc.resid ** 2)
        beta1_b_list.append(beta1_b)
        beta2_b_list.append(beta2_b)
        sigma_b_list.append(sigma_b)
        print('b=',b)
    beta1_mean_list.append(np.mean(beta1_b_list))
    beta1_std_list.append(np.std(beta1_b_list))
    beta2_mean_list.append(np.mean(beta2_b_list))
    beta2_std_list.append(np.std(beta2_b_list))
    sigma_mean_list.append(np.mean(sigma_b_list))
    sigma_std_list.append(np.std(sigma_b_list))
print(beta1_mean_list, beta1_std_list)
print(beta2_mean_list, beta2_std_list)
print(sigma_mean_list, sigma_std_list)

alpha = 0.95 # 0.9 0.95, 0.975, 0.995 (0.8, 0.9, 0.95, 0.99)
beta1_lb_list = np.add(beta1_mean_list, [-norm.ppf(alpha, loc=0, scale=1)*i for i in beta1_std_list])
beta1_ub_list = np.add(beta1_mean_list, [norm.ppf(alpha, loc=0, scale=1)*i for i in beta1_std_list])
beta2_lb_list = np.add(beta2_mean_list, [-norm.ppf(alpha, loc=0, scale=1)*i for i in beta2_std_list])
beta2_ub_list = np.add(beta2_mean_list, [norm.ppf(alpha, loc=0, scale=1)*i for i in beta2_std_list])
sigma_lb_list = np.add(sigma_mean_list, [-norm.ppf(alpha, loc=0, scale=1)*i for i in sigma_std_list])
sigma_ub_list = np.add(sigma_mean_list, [norm.ppf(alpha, loc=0, scale=1)*i for i in sigma_std_list])
beta1_in = 0
beta2_in = 0
sigma_in = 0
for i in range(trial):
    if beta1_lb_list[i] < 0.5 and beta1_ub_list[i] > 0.5:
        beta1_in += 1
    if beta2_lb_list[i] < 0.5 and beta2_ub_list[i] > 0.5:
        beta2_in += 1
    if sigma_lb_list[i] < 1 and sigma_ub_list[i] > 1:
        sigma_in += 1
print(beta1_in)
print(beta2_in)
print(sigma_in)


Optimization terminated successfully.
         Current function value: 0.909950
         Iterations: 63
         Function evaluations: 116
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0609      0.499     -0.122      0.903      -1.038       0.916
Z              0.6720      0.113      5.937      0.000       0.450       0.894
Log(Sigma)    -0.0914      0.135     -0.677      0.498      -0.356       0.173
Optimization terminated successfully.
         Current function value: -18.155899
         Iterations: 245
         Function evaluations: 457


c:\Users\NTUpo\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


b= 0
Optimization terminated successfully.
         Current function value: 0.780614
         Iterations: 69
         Function evaluations: 121
b= 1
Optimization terminated successfully.
         Current function value: 0.842634
         Iterations: 62
         Function evaluations: 116
b= 2
Optimization terminated successfully.
         Current function value: 0.864864
         Iterations: 61
         Function evaluations: 112
b= 3
Optimization terminated successfully.
         Current function value: 0.835933
         Iterations: 76
         Function evaluations: 131
b= 4
Optimization terminated successfully.
         Current function value: 0.785519
         Iterations: 78
         Function evaluations: 133
b= 5
Optimization terminated successfully.
         Current function value: 0.940365
         Iterations: 73
         Function evaluations: 136
b= 6
Optimization terminated successfully.
         Current function value: 0.774285
         Iterations: 70
         Function evaluatio

c:\Users\NTUpo\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


b= 3
Optimization terminated successfully.
         Current function value: 0.758054
         Iterations: 75
         Function evaluations: 132
b= 4
Optimization terminated successfully.
         Current function value: 0.694704
         Iterations: 60
         Function evaluations: 110
b= 5
Optimization terminated successfully.
         Current function value: 0.703708
         Iterations: 53
         Function evaluations: 97
b= 6
Optimization terminated successfully.
         Current function value: 0.778431
         Iterations: 71
         Function evaluations: 127
b= 7
Optimization terminated successfully.
         Current function value: 0.905277
         Iterations: 65
         Function evaluations: 119
b= 8
Optimization terminated successfully.
         Current function value: 0.845912
         Iterations: 64
         Function evaluations: 113
b= 9
Optimization terminated successfully.
         Current function value: 0.995515
         Iterations: 67
         Function evaluation

c:\Users\NTUpo\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


b= 6
Optimization terminated successfully.
         Current function value: 0.887350
         Iterations: 61
         Function evaluations: 110
b= 7
Optimization terminated successfully.
         Current function value: 0.877212
         Iterations: 77
         Function evaluations: 134
b= 8
Optimization terminated successfully.
         Current function value: 0.894209
         Iterations: 58
         Function evaluations: 107
b= 9
Optimization terminated successfully.
         Current function value: 1.012234
         Iterations: 65
         Function evaluations: 115
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0995      0.366      3.002      0.003       0.382       1.817
Z              0.3404      0.091      3.731      0.000       0.162       0.519
Log(Sigma)    -0.1143      0.139     -0.825      0.409      -0.386       0.157
Optimization terminated succe

c:\Users\NTUpo\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


b= 9
Optimization terminated successfully.
         Current function value: 0.971926
         Iterations: 46
         Function evaluations: 86
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3297      0.283      4.695      0.000       0.775       1.885
Z              0.3131      0.074      4.212      0.000       0.167       0.459
Log(Sigma)    -0.2413      0.128     -1.878      0.060      -0.493       0.011
Optimization terminated successfully.
         Current function value: 0.520622
         Iterations: 49
         Function evaluations: 91
b= 0
Optimization terminated successfully.
         Current function value: 0.466496
         Iterations: 51
         Function evaluations: 92
b= 1
Optimization terminated successfully.
         Current function value: 0.557436
         Iterations: 54
         Function evaluations: 100
b= 2
Optimization terminated successf